In [1]:
#####################################################################
########################### vATISLoad.py ############################
#####################################################################
import subprocess, sys, os, time, json, re, uuid, ctypes
from datetime import datetime

# pip uninstall -y pyautogui pyperclip pygetwindow pywin32 pywinutils psutil
import importlib.util as il
if None in [il.find_spec('pyautogui'), il.find_spec('pyperclip'), \
            il.find_spec('pygetwindow'), il.find_spec('win32api'), \
            il.find_spec('psutil'), il.find_spec('requests'), \
            il.find_spec('pyscreeze')]:
    subprocess.check_call([sys.executable, '-m', 'pip', 
                       'install', 'pyautogui']);
    subprocess.check_call([sys.executable, '-m', 'pip', 
                           'install', 'pyperclip']);
    subprocess.check_call([sys.executable, '-m', 'pip', 
                           'install', 'pygetwindow']);
    subprocess.check_call([sys.executable, '-m', 'pip', 
                           'install', 'pywinutils']);
    subprocess.check_call([sys.executable, '-m', 'pip', 
                           'install', 'psutil']);
    subprocess.check_call([sys.executable, '-m', 'pip', 
                           'install', 'requests']);
    subprocess.check_call([sys.executable, '-m', 'pip', 
                           'install', 'pyscreeze']);
    subprocess.check_call([sys.executable, '-m', 'pip', 
                           'install', '--upgrade', 'Pillow']);
    os.system('cls')
else:
    os.system('cls')
    
import requests, pyautogui, psutil, pyperclip, pygetwindow as gw
from win32 import win32api, win32gui, win32gui, win32process
from win32.lib import win32con

scale_factor = ctypes.windll.shcore.GetScaleFactorForDevice(0) / 100

In [2]:
tab_sizes = {'small': 70, 'large': 95, 'small_con': 90, 'large_con': 118}

def determine_active_profile():
    crc_profiles = os.getenv('LOCALAPPDATA') + '\\CRC\\Profiles'
    crc_name = ''
    crc_lastused_time = '2020-01-01T08:00:00'
    for filename in os.listdir(crc_profiles):
        if filename.endswith('.json'): 
            file_path = os.path.join(crc_profiles, filename)
            with open(file_path, 'r') as f:
                data = json.load(f)
                dt1 = datetime.strptime(crc_lastused_time, '%Y-%m-%dT%H:%M:%S')
                dt2 = datetime.strptime(data['LastUsedAt'].split('.')[0], '%Y-%m-%dT%H:%M:%S')
                if dt2 > dt1:
                    crc_lastused_time = data['LastUsedAt'].split('.')[0]
                    crc_name = data['Name']

    vatis_profiles = os.getenv('LOCALAPPDATA') + '\\org.vatsim.vatis\\Profiles'
    for filename in os.listdir(vatis_profiles):
        if filename.endswith('.json'): 
            file_path = os.path.join(vatis_profiles, filename)
            with open(file_path, 'r') as f:
                data = json.load(f)
                vatis_abr = data['name'].split('(')[1][0:3]
                if vatis_abr in crc_name:
                    return data['name']
    return ''

def get_active_profile_position(profile):
    vatis_profiles = os.getenv('LOCALAPPDATA') + '\\org.vatsim.vatis\\Profiles'
    profile_names = []
    for filename in os.listdir(vatis_profiles):
        if filename.endswith('.json'): 
            file_path = os.path.join(vatis_profiles, filename)
            with open(file_path, 'r') as f:
                data = json.load(f)
                profile_names.append(data['name'])

    if profile in profile_names:
        return profile_names.index(profile)
    return -1

def open_vATIS():
    os.system('taskkill /f /im vATIS.exe 2>nul 1>nul')
    exe = os.getenv('LOCALAPPDATA') + '\\org.vatsim.vatis\\current\\vATIS.exe'
    subprocess.Popen(exe);
    
    for i in range(0, 50): 
        vatis_open = False
        for window in gw.getAllWindows():
            if 'vATIS Profiles' in window.title:
                vatis_open = True
        if vatis_open:
            time.sleep(0.5)
            return
        else:
            time.sleep(0.1)

def get_win(exe_name, window_title):
    for window in gw.getAllWindows():
        hwnd = window._hWnd
        thread_id, process_id = win32process.GetWindowThreadProcessId(hwnd)
        process = psutil.Process(process_id)
        process_name = process.name()
        process_path = process.exe()
        if exe_name in process_path:
            if window.title == window_title:
                return window

def click_xy(xy, win, d=0, sf=True):
    x, y = xy
    if sf:
        x *= scale_factor
        y *= scale_factor
    x += win.left
    y += win.top
    time.sleep(d)
    pyautogui.moveTo(x, y)
    pyautogui.click()

def get_online_atises(airports):
    url = "https://data.vatsim.net/v3/vatsim-data.json"
    response = requests.get(url)
    data = response.json()

    online_atises = []
    for atis in data['atis']:
        if atis['callsign'].replace('_ATIS', '') in airports:
            online_atises.append(atis['callsign'].replace('_ATIS', ''))
    return online_atises

def read_profile(profile):
    vatis_profiles = os.getenv('LOCALAPPDATA') + '\\org.vatsim.vatis\\Profiles'
    for filename in os.listdir(vatis_profiles):
        if filename.endswith('.json'): 
            file_path = os.path.join(vatis_profiles, filename)
            with open(file_path, 'r') as f:
                data = json.load(f)
                if data['name'] == profile:
                    return data

def get_stations(data):
    stations = []
    for station in data['stations']:
        s = station['identifier']
        if station['atisType'] == 'Departure':
            s += '_D'
        elif station['atisType'] == 'Arrival':
            s += '_A'
        stations.append(s)

    return stations

def get_station_order(stations, online_atises):
    offline_atises = list(set(stations) - set(stations).intersection(online_atises))

    left_pad = 20
    for atis in online_atises:
        if len(atis) == 4:
            left_pad += tab_sizes['small_con']
        else:
            left_pad += tab_sizes['large_con']
    
    return sorted(offline_atises), left_pad

In [4]:
active_profile = determine_active_profile()
if len(active_profile) == 0:
    print('Active profile not found. TODO')
    time.sleep(10)
    sys.exit()

# Open vATIS and select first profile
open_vATIS()
pyautogui.PAUSE = 0.001
win = get_win('vATIS.exe', 'vATIS Profiles')
click_xy([0, 0], win)
pyautogui.press('tab')

# Select active profile
for i in range(0, get_active_profile_position(active_profile)):
    pyautogui.press('down')
pyautogui.press('enter')
with pyautogui.hold('shift'):
    pyautogui.press('tab', presses=5)
pyautogui.press('enter')

data = read_profile(active_profile)
stations = get_stations(data)
online_atises = get_online_atises(stations)

for i in range(0, 50):
    try:
        win = get_win('vATIS.exe', 'vATIS')
    except Exception as ignored:
        time.sleep(0.1)
        pass

time.sleep(2.5)
load_atis('KMIA_D', stations, data)
load_atis('KRSW', stations, data)

NameError: name 'load_atis' is not defined

In [14]:
def load_atis(station, stations, data):
    station_order, left_pad = get_station_order(stations, online_atises)

    if station not in station_order:
        return

    # Determine where to click to select profile
    for s in station_order:
        if s == station:
            if len(station) == 4:
                left_pad += tab_sizes['small'] / 2
            else:
                left_pad += tab_sizes['large'] / 2
            break
        else:
            if len(s) == 4:
                left_pad += tab_sizes['small']
            else:
                left_pad += tab_sizes['large']
    
    station_data = {}
    for elem in data['stations']:
        if station[0:4] in elem['identifier']:
            if len(station) == 4:
                station_data = elem
                break
            elif (station[5] == 'D' and elem['atisType'] == 'Departure') or \
                (station[5] == 'A' and elem['atisType'] == 'Arrival'):
                station_data = elem
                break

    # Make sure profile has D-ATIS preset
    if station_data['presets'][0]['name'] != 'D-ATIS':
        return

    # Select profile
    click_xy([left_pad, 100], win, sf=False)

    # Select D-ATIS preset
    click_xy([500, 500], win, sf=False, d=0.1)
    click_xy([500, 550], win, sf=False, d=0.1)

    # Click AIRPORT CONDITIONS field
    click_xy([335, 390], win, sf=False)

    # Click NOTAMS field
    # click_xy([940, 390], win, sf=False)

    # Select profile again and connect ATIS
    click_xy([left_pad, 100], win, sf=False)
    with pyautogui.hold('shift'):
        pyautogui.press('tab', presses=7)
    pyautogui.press('enter')

    online_atises.append(station)

online_atises = get_online_atises(stations)

for i in range(0, 50):
    try:
        win = get_win('vATIS.exe', 'vATIS')
    except Exception as ignored:
        time.sleep(0.1)
        pass

time.sleep(2.5)
load_atis('KMIA_D', stations, data)
load_atis('KRSW', stations, data)

In [ ]:
win = get_win('vATIS.exe', 'vATIS')

# Determine mouse position (and color) on held left click
def mouse_position(color=False):
    prev_xy = [-99999, -99999]
    for i in range(0, 20):
        time.sleep(1)
        if win32api.GetAsyncKeyState(0x01) >= 0:
            continue
        x, y = pyautogui.position()
        win_bound = [win.left, win.top]
        out = x - win_bound[0], y - win_bound[1]
        if out[0] != prev_xy[0] or out[1] != prev_xy[1]:
            if not color:
                print(out)
            else:
                print(out, pyautogui.pixel(x, y))
            prev_xy = out[:]
            
mouse_position()

In [ ]:
raise Exception

In [ ]:
def get_atis(ident):
    atis_type = 'C'
    if '/' in ident:
        ident, atis_type = ident.split('/')
    if len(ident) == 3:
        ident = 'K' + ident
    url = 'https://datis.clowd.io/api/' + ident

    atis_info, code = [], ''
    atis_data = json.loads(requests.get(url).text)
    if 'error' in atis_data:
        return [], ''
    
    for n in range(0, len(atis_data)):
        datis = atis_data[n]['datis']
        if atis_type == 'C' and n == 0:
            code = atis_data[n]['code']
        elif atis_type == 'D' and atis_data[n]['type'] == 'dep':
            code = atis_data[n]['code']
        elif atis_type == 'A' and atis_data[n]['type'] == 'arr':
            code = atis_data[n]['code']

        datis = re.sub('.*INFO [A-Z] [0-9][0-9][0-9][0-9]Z. ', '', datis)
        datis = '. '.join(datis.split('. ')[1:])
        datis = re.sub(' ...ADVS YOU HAVE.*', '', datis)
        datis = datis.replace('...', '/./').replace('..', '.') \
            .replace('/./', '...').replace('  ', ' ').replace(' . ', '. ') \
            .replace(', ,', ',').replace(' ; ', '; ').replace(' .,', ' ,') \
            .replace(' , ', ', ').replace('., ', ', ').replace('&amp;', '&')

        info = []
        if 'NOTAMS' in datis:
            info = datis.split('NOTAMS... ')
        elif 'NOTICE TO AIR MISSIONS. ' in datis:
            info = datis.split('NOTICE TO AIR MISSIONS. ')
        else:
            info = [datis, '']
        
        if n == 0:
            atis_info = info[:]
        else:
            if atis_type == 'A':
                atis_info[0] = re.sub(r'\s+', ' ', atis_info[0])
            elif atis_type == 'D':
                atis_info[0] = re.sub(r'\s+', ' ', info[0])
            else:
                atis_info[0] = re.sub(r'\s+', ' ', 
                                      info[0] + ' ' + atis_info[0])

    return atis_info, code

In [ ]:
# Center command prompt
for win in gw.getAllWindows():
    _, process_id = win32process.GetWindowThreadProcessId(win._hWnd)
    process = psutil.Process(process_id)
    if 'py.exe' in win.title or (win.title == 'vATIS' and 'vATIS.exe' not in process.exe()): 
        screen_dim = [win32api.GetSystemMetrics(0), \
            win32api.GetSystemMetrics(1)]
        win.moveTo(int((screen_dim[0] - win.size[0]) / 2) - 60, \
            int((screen_dim[1] - win.size[1]) / 2))

# Profile selection
print('=========== vATISLoad ===========')
profiles, TIMEOUT = read_config()
for i in range(0, len(profiles)):
    facility = list(profiles.keys())[i]
    airports = ', '.join(list(profiles.values())[i])
    print(f'({i}) {facility} - {airports}')
    
if len(profiles) == 0:
    print('No vATISLoadConfig.json file found!')
    print('Add profiles to create a configuration file.\n')
    
print('(A) Add new profile')

for i in range(0, 100):
    idx = input('\nProfile: ')
    if idx.isdigit():
        idx = int(idx)
        if idx < len(profiles):
            break
    elif idx.upper() == 'A':
        os.system('cls')
        print('=========== vATISLoad ===========')
        print('Input the vATIS facility name')
        print('e.g. \'Oakland ARTCC (ZOA)\'')
        print('e.g. \'ZOA\'')
        facility = input('\nFacility: ')
        os.system('cls')
        print('=========== vATISLoad ===========')
        print('Input airports separated by commas')
        print('DEP/ARR ATISes - add \'/D\' or \'/A\'')
        print('e.g. \'MIA/D, MIA/A, FLL, TPA, RSW\'')
        airports = input('\nAirports: ')
        add_profile(facility, airports)
        profiles, TIMEOUT = read_config()
        idx = len(profiles) - 1
        break
    print(f'Invalid input! Selection must be a number ' \
          + f'between 0 and {len(profiles) - 1}.')

os.system('cls')
PROFILE = list(profiles.keys())[idx]
AIRPORTS = list(profiles.values())[idx]

# Create missing D-ATIS presets
check_datis_profile(PROFILE)

print('=========== vATISLoad ===========')
print(f'[{PROFILE}]')

# Open vATIS
open_vATIS()
pyautogui.PAUSE = 0.001

# Center 'vATIS Profiles' window and bring to foreground
win = center_win('vATIS.exe', 'vATIS Profiles')

# Select profile chosen above
win_bound = [win.left, win.top]
n_profile = get_profile_pos(PROFILE, sort=True)
if n_profile == -1:
    print('Selected profile not found!')
    print('Ensure facility name is unique/exists\n')
    print('Rename profile in \'vATISLoadConfig.json\'')
    print('See the README on GitHub for more info')
    time.sleep(10)
    sys.exit()
elif n_profile > 18:
    print('You must have <= 19 vATIS profiles')
    print('vATISLoad does not support > 19 currently')
    time.sleep(10)
    sys.exit()
loc_profile = [90, 40 + 14 * n_profile]

click_xy(loc_profile, win)
pyautogui.press('enter')

time.sleep(1)

# Center 'vATIS' window and bring to foreground
win = center_win('vATIS.exe', 'vATIS')

for ident in AIRPORTS:
    # Select tab for specified airport
    tab = get_tab(ident, PROFILE)
    if tab == -1:
        print(f'{ident} NOT FOUND.')
        continue
    loc_tab = [38.6 + 53.6 * tab, 64]
    click_xy(loc_tab, win)
    
    # Select first preset
    click_xy([400, 330], win)
    pyautogui.press(['up', 'enter'])
    
    # Get D-ATIS
    atis, code = get_atis(ident)
    
    # Enter ARPT COND
    if len(atis) > 0:
        click_xy([200, 250], win)
        pyautogui.hotkey('ctrl', 'a')
        pyautogui.press('backspace')
        pyperclip.copy(atis[0])
        pyautogui.hotkey('ctrl', 'v')
        click_xy([40, 295], win)
    
    # Enter NOTAMS
    if len(atis) > 1:
        click_xy([600, 250], win)
        pyautogui.hotkey('ctrl', 'a')
        pyautogui.press('backspace')
        pyperclip.copy(atis[1])
        pyautogui.hotkey('ctrl', 'v')
        click_xy([415, 295], win)
        
    if len(code) == 0 or len(atis[0]) == 0:
        print(f'{ident.upper()} - UN')
        continue
    
    # Connect ATIS
    click_xy([720, 330], win)
    state = ''
    for i in range(0, int(10 * TIMEOUT)):
        pix_x = int(scale_factor * 118)
        pix_y = int(scale_factor * 104)
        pix = pyautogui.pixel(win.left + pix_x, win.top + pix_y)
        # Check if METAR loads (white 'K')
        if pix[0] >= 248 or pix[1] >= 248:
            state = 'CON'
            break
        # Check if ATIS is already connected (red 'N')
        elif pix[0] >= 220 and pix[0] <= 230:
            state = 'ON'
            break
        time.sleep(.1)
    # Set ATIS code
    for i in range(0, char_position(code)):
        click_xy([62, 130], win)
    if state == 'CON':
        print(f'{ident.upper()} - {code}')
    elif state == 'ON':
        print(f'{ident.upper()} - OL/{code}')
    else:
        if len(code) > 0:
            print(f'{ident.upper()} - UN/{code}')
        else:  
            print(f'{ident.upper()} - UN')

pyperclip.copy('')
time.sleep(3)
win32gui.ShowWindow(win._hWnd, win32con.SW_MINIMIZE);